In [2]:
import math, pickle, glob
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.pylab as pl
from scipy.stats import linregress

In [20]:
one_label_one_data = False
workspace = 'oja'
label     = 'oja_finetune2base'
figlabel = workspace

targeton = False

label_dir = '../data/{}/{}/'.format(workspace,label)
wild_dir  = '../data/{}/{}/*.pickle'.format(workspace,label)

list_for_lpwise_dicts = []

count = 0

label_list = glob.glob(label_dir)
for label in label_list:
    data_list = glob.glob(label+'*.pickle')
    for data_path in data_list:
        with open(data_path, 'rb') as f:
            hyperdict  = pickle.load(f)
            lpwise_dict = pickle.load(f)
            list_for_lpwise_dicts.append((hyperdict, lpwise_dict))
            count += 1 
        if one_label_one_data:
            break
    
print('{} data were loaded.'.format(count))

algo = hyperdict['algo']
T, T_save, loop, k, p = hyperdict['T'], hyperdict['T_save'] ,hyperdict['loop'], hyperdict['k'], hyperdict['p']
sigma, delta = hyperdict['sigma'], hyperdict['delta']
learning_param_list = np.asarray(hyperdict['lp_list'])

#Gamma selection
allgamma = [x[0]['gamma'] for x in list_for_lpwise_dicts]

#target = np.asarray([0.0, 0.006, 0.01, 0.02, 0.04, 0.06, 0.08, 0.1]) * delta
target = np.asarray([0.0, 0.00001, 0.00003, 0.00005]) * delta

targetidx = []
for idx, gamma in enumerate(allgamma):
    if np.any(np.isclose(gamma, target)) or (not targeton):
        targetidx.append(idx)
list_for_lpwise_dicts = [list_for_lpwise_dicts[idx] for idx in targetidx]

list_for_lpwise_dicts = sorted(list_for_lpwise_dicts, key=lambda v: (v[0]['p'],-v[0]['delta'],v[0]['gamma'],v[0]['k']))

        
print([x[0]['gamma'] for x in list_for_lpwise_dicts])

if algo == 'noisy':
    noisy_lpwise_dicts = list_for_lpwise_dicts
    noisy_param_list   = learning_param_list
elif algo == 'oja':
    oja_lpwise_dicts = list_for_lpwise_dicts
    oja_param_list   = learning_param_list
else:
    raise NotImplementedError

print('Hyper Dictionary Sample: ')
for k,v in hyperdict.items():
    print('{}: {}'.format(k,v))

19 data were loaded.
[1.05e-05, 1.1e-05, 1.15e-05, 1.21e-05, 1.27e-05, 1.33e-05, 1.4e-05, 1.48e-05, 1.56e-05, 1.65e-05, 1.76e-05, 1.87e-05, 2.18e-05, 2.35e-05, 2.53e-05, 2.74e-05, 3.26e-05, 3.58e-05, 4.42e-05]
Hyper Dictionary Sample: 
T: 144000
T_save: 100
loop: 3
k: 5
p: 100
sigma: 0.15
gamma: 1.87e-05
delta: 1.0
algo: oja
lp_list: [5.00000000e-01 3.33333333e-01 1.25000000e-01 1.00000000e-01
 5.00000000e-02 3.33333333e-02 2.50000000e-02 1.66666667e-02
 1.25000000e-02 1.00000000e-02 5.00000000e-03 3.33333333e-03
 2.50000000e-03 1.66666667e-03 1.25000000e-03 1.00000000e-03
 8.33333333e-04 6.66666667e-04 5.55555556e-04 5.00000000e-04
 3.33333333e-04 2.50000000e-04 1.66666667e-04 1.25000000e-04
 1.04166667e-04]
label: oja_finetune2base
rotate_type: True


In [21]:
targetexist = True
#target = np.asarray([0.00625, 0.0085, 0.0095]) * delta

target = allgamma

dictlp = {}
for c, (hyperdict, datadict_list) in enumerate(list_for_lpwise_dicts):
    
    gamma = hyperdict['gamma']
    
    if not np.any(np.isclose(gamma, target)) or np.isclose(gamma, 0.0):
        continue
    
    dist_array = np.zeros((loop,len(learning_param_list)))
    
    dist_vec_list = []
    for datadict in datadict_list:
        dist_vec = np.zeros((1,len(learning_param_list)))
        for jdx,lp in enumerate(learning_param_list):
            dist_vec[0,jdx] = np.average(datadict[lp][-15:])
            
        dist_vec_list.append(dist_vec)
        
    dist_array = np.vstack(dist_vec_list) 

    avg = np.average(dist_array,axis=0)
    std = np.std(dist_array,axis=0)
    
    arg = np.argmin(avg)
    
    if 0 < arg < len(learning_param_list)-1:
        dictlp[gamma] = (max(arg-2,0),min(arg+2,21))
        
    
print(dictlp)

with open(algo+'_lpidxdict2.pickle', 'wb') as f:
    pickle.dump(dictlp, f, pickle.HIGHEST_PROTOCOL)

{1.05e-05: (17, 21), 1.1e-05: (16, 20), 1.15e-05: (15, 19), 1.21e-05: (16, 20), 1.27e-05: (15, 19), 1.33e-05: (15, 19), 1.4e-05: (16, 20), 1.48e-05: (15, 19), 1.56e-05: (16, 20), 1.65e-05: (14, 18), 1.76e-05: (15, 19), 1.87e-05: (15, 19), 2.18e-05: (14, 18), 2.35e-05: (13, 17), 2.53e-05: (14, 18), 2.74e-05: (14, 18), 3.26e-05: (12, 16), 3.58e-05: (12, 16), 4.42e-05: (11, 15)}
